In [1]:
!pip install transformers datasets seqeval


In [2]:
!pip install evaluate


In [3]:
file_path = "C:/Users/ibsan/Desktop/TenX/week-5/data/labeled_messages.conll"

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, Features, Value, Sequence, ClassLabel
import evaluate  # Updated import for metrics
import numpy as np

# Function to parse the .conll file
def read_conll_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    tokens = []
    labels = []
    current_tokens = []
    current_labels = []

    for line in lines:
        line = line.strip()
        if line == "":  # End of a sentence
            if current_tokens:
                tokens.append(current_tokens)
                labels.append(current_labels)
                current_tokens = []
                current_labels = []
        else:
            parts = line.split()  # Assuming the format is: token [tab] label
            if len(parts) == 2:
                current_tokens.append(parts[0])
                current_labels.append(parts[1])

    # Add the last sentence if the file doesn't end with a blank line
    if current_tokens:
        tokens.append(current_tokens)
        labels.append(current_labels)

    return {"tokens": tokens, "labels": labels}

# Load your .conll file
file_path = "C:/Users/ibsan/Desktop/TenX/week-5/data/labeled_messages.conll"
data = read_conll_file(file_path)



In [5]:
# Get unique labels from the dataset
unique_labels = sorted(list(set(label for sublist in data["labels"] for label in sublist)))

# Define the features for the dataset
features = Features({
    "tokens": Sequence(Value("string")),
    "labels": Sequence(ClassLabel(names=unique_labels)),  # Treat labels as ClassLabel
})

# Convert to Hugging Face Dataset with explicit features
dataset = Dataset.from_dict(data, features=features)

# Split into train and validation sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)

print(dataset)

# Function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=512,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:  # Special tokens (e.g., [CLS], [SEP])
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # New word
                label_ids.append(label[word_idx])
            else:  # Same word (subword)
                label_ids.append(-100)  # Use -100 to ignore subwords in the loss function
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 40
    })
    test: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 10
    })
})


In [ ]:

# # Define the models to compare
# models_to_compare = {
#     "xlm-roberta": "xlm-roberta-base",
#     "distilbert": "distilbert-base-multilingual-cased",
#     "mbert": "bert-base-multilingual-cased",
# }

# custom_base_dir = "C:/Users/ibsan/Desktop/TenX/week-5/model_output/fine-tuned-model"

# # Fine-tune each model
# for model_name, model_checkpoint in models_to_compare.items():
#     print(f"Fine-tuning {model_name}...")

#     # Load the tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

#     # Tokenize the dataset
#     tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

#     # Load the model
#     model = AutoModelForTokenClassification.from_pretrained(
#         model_checkpoint,
#         num_labels=len(unique_labels)  # Number of unique labels
#     )

#     # Define training arguments
#     training_args = TrainingArguments(
#         output_dir=f"./{model_name}-fine-tuned",
#         evaluation_strategy="epoch",
#         learning_rate=2e-5,
#         per_device_train_batch_size=16,
#         per_device_eval_batch_size=16,
#         num_train_epochs=3,
#         weight_decay=0.01,
#         save_strategy="epoch",
#         logging_dir=f"./{model_name}-logs",
#         logging_steps=10,
#         report_to="none",
#     )

#     # Define the Trainer
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=tokenized_dataset["train"],
#         eval_dataset=tokenized_dataset["test"],
#         tokenizer=tokenizer,
#     )

#     # Fine-tune the model
#     trainer.train()

#     # Save the fine-tuned model
#     trainer.save_model(f"./{model_name}-fine-tuned")
#     tokenizer.save_pretrained(f"./{model_name}-fine-tuned")

#     print(f"{model_name} fine-tuning completed and saved successfully!")

#     # Load the accuracy metric
#     accuracy_metric = evaluate.load("accuracy")

#     # Evaluate the model
#     results = trainer.evaluate()
#     print(f"{model_name} Evaluation results: {results}")

#     # Calculate accuracy using the metric
#     eval_accuracy = accuracy_metric.compute(predictions=results["predictions"], references=results["labels"])
#     print(f"{model_name} Accuracy: {eval_accuracy}")

Fine-tuning xlm-roberta...


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ibsan\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ibsan\AppData\Local\Temp\ipykernel_66456\3867300287.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyboardInterrupt: 

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# Define the models to compare
models_to_compare = {
    "xlm-roberta": "xlm-roberta-base",
    "distilbert": "distilbert-base-multilingual-cased",
    "mbert": "bert-base-multilingual-cased",
}

# Define a custom base directory for saving models
custom_base_dir = "/path/to/your/custom/directory"

# Fine-tune each model
for model_name, model_checkpoint in models_to_compare.items():
    print(f"Fine-tuning {model_name}...")

    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    # Tokenize the dataset
    tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

    # Load the model
    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        num_labels=len(unique_labels)  # Number of unique labels
    )

    # Define training arguments with custom output directory
    training_args = TrainingArguments(
        output_dir=f"{custom_base_dir}/{model_name}-fine-tuned",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        save_strategy="epoch",
        logging_dir=f"{custom_base_dir}/{model_name}-logs",
        logging_steps=10,
        report_to="none",
    )

    # Define the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
    )

    # Fine-tune the model
    trainer.train()

    # Save the fine-tuned model to the custom directory
    model_save_path = f"{custom_base_dir}/{model_name}-fine-tuned"
    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    print(f"{model_name} fine-tuning completed and saved successfully at {model_save_path}!")

    # Load the accuracy metric
    accuracy_metric = evaluate.load("accuracy")

    # Evaluate the model and get predictions
    predictions, labels, metrics = trainer.predict(tokenized_dataset["test"])

    # Convert logits to predicted class indices
    predictions = np.argmax(predictions, axis=-1)

    # Flatten predictions and labels
    flat_predictions = predictions.flatten()
    flat_labels = labels.flatten()

    # Filter out -100 values (ignored labels)
    mask = flat_labels != -100
    filtered_predictions = flat_predictions[mask]
    filtered_labels = flat_labels[mask]

    # Calculate accuracy using the metric
    eval_accuracy = accuracy_metric.compute(predictions=filtered_predictions, references=filtered_labels)
    print(f"{model_name} Accuracy: {eval_accuracy}")

Fine-tuning xlm-roberta...


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ibsan\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ibsan\AppData\Local\Temp\ipykernel_66456\3303335815.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,2.063426
2,No log,1.808048
3,No log,1.670512


...

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ibsan\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ibsan\AppData\Local\Temp\ipykernel_66456\3303335815.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.006708
2,No log,0.828552
3,No log,0.782473


mbert fine-tuning completed and saved successfully at /path/to/your/custom/directory/mbert-fine-tuned!


mbert Accuracy: {'accuracy': 0.7683397683397684}
